In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/학부연구생/C프로젝트/C프로젝트 최종정리/first_cleaned_data.csv' )

In [ ]:
df

,EXT1,EXT2,EXT3,EXT4,EXT5,EXT6,EXT7,EXT8,EXT9,EXT10,...,OPN1,OPN2,OPN3,OPN4,OPN5,OPN6,OPN7,OPN8,OPN9,OPN10
0,4,5,5,4,5,5,5,4,4,5,...,5,5,4,5,4,5,5,3,4,5
1,3,1,3,2,3,3,2,1,1,1,...,1,4,4,4,3,5,4,2,5,3
2,2,3,4,2,3,4,1,3,2,1,...,5,5,2,5,4,4,5,3,4,4
3,2,4,2,3,4,4,2,2,1,2,...,4,4,5,4,3,5,4,4,3,3
4,3,3,3,3,5,3,3,1,3,2,...,5,5,5,5,5,5,5,3,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813221,4,4,4,3,4,3,3,3,3,3,...,2,4,4,3,4,4,4,2,2,4
813222,4,3,4,3,3,3,4,2,3,3,...,4,5,5,5,5,5,3,4,5,4
813223,4,4,4,3,5,5,4,4,4,2,...,5,5,5,5,4,5,5,5,4,5
813224,2,2,3,2,2,4,1,2,2,2,...,5,4,4,4,3,4,4,5,5,3


In [ ]:
# 요인별 문항 리스트 정의
factors = {
    "factor1": [f"EXT{i}" for i in range(1, 11)],
    "factor2": [f"EST{i}" for i in range(1, 11)],
    "factor3": [f"AGR{i}" for i in range(1, 11)],
    "factor4": [f"CSN{i}" for i in range(1, 11)],
    "factor5": [f"OPN{i}" for i in range(1, 11)],
}


In [ ]:

# item-total correlation 함수
def get_reduced_index(df_subset):
    high_index = df_subset.corr(method='pearson').sum(axis= 1).sort_values(ascending=False)[:6].index
    return high_index
    # 여기서 상관계수가 가장 높은 상위 5개 index가 뽑혀 나옴.

# 크론바흐 알파 계산
def cronbach_alpha(df_subset):
    item_vars = df_subset.var(axis=0, ddof=1)
    total_var = df_subset.sum(axis=1).var(ddof=1)
    n_items = df_subset.shape[1]
    return (n_items / (n_items - 1)) * (1 - item_vars.sum() / total_var)


# 각 요인별로 크론바흐 알파 계산

In [ ]:
factors_cronbach_alphas = []
for factor_name, items in factors.items():
    df_factor = df[items]
    alpha = cronbach_alpha(df_factor)
    print(f"{factor_name} 크론바흐 알파: {alpha:.3f}")
    factors_cronbach_alphas.append(alpha)

print(f'전체 크론바흐 알파 평균 : {np.mean(factors_cronbach_alphas):.3f}')

factor1 크론바흐 알파: 0.906
factor2 크론바흐 알파: 0.881
factor3 크론바흐 알파: 0.848
factor4 크론바흐 알파: 0.832
factor5 크론바흐 알파: 0.815
전체 크론바흐 알파 평균 : 0.856


In [ ]:
high_five_index

Index(['OPN10', 'OPN2', 'OPN5', 'OPN1', 'OPN3'], dtype='object')

In [ ]:
# 문항단축 수행 (각 요인별로 상위 6개만 선택)
alpha_afters = []
df_list = []
for factor_name, items in factors.items():
    df_factor = df[items]
    high_six_index = get_reduced_index(df_factor)
    alpha_before = cronbach_alpha(df_factor)
    # 상위 6개 문항 선택
    #하이 파이브가 여기 들어갈 것...
    df_selected = df_factor[high_six_index]
    alpha_after = cronbach_alpha(df_selected)
    df_selected.reset_index(drop=True, inplace=True)
    df_list.append(df_selected)

    print(f"\n[{factor_name}]")
    print("초기 문항 수:", len(items))
    print("선택된 문항 수:", len(df_selected.columns))
    print("alpha: {:.3f} → {:.3f}".format(alpha_before, alpha_after))

    alpha_afters.append(alpha_after)

df_reduced = pd.concat(df_list,axis= 1 )

# 전체 알파 계산
overall_alpha = cronbach_alpha(df_reduced)
print("\n축소된 전체 척도 문항 수:", df_reduced.shape[1])
print(f'축소 전 전체 크론바흐 알파 평균 : {np.mean(factors_cronbach_alphas):.3f}')
print(f"축소 후  크론바흐 알파 평균: {np.mean(alpha_afters):.3f}")


[factor1]
초기 문항 수: 10
선택된 문항 수: 6
alpha: 0.906 → 0.882

[factor2]
초기 문항 수: 10
선택된 문항 수: 6
alpha: 0.881 → 0.868

[factor3]
초기 문항 수: 10
선택된 문항 수: 6
alpha: 0.848 → 0.847

[factor4]
초기 문항 수: 10
선택된 문항 수: 6
alpha: 0.832 → 0.806

[factor5]
초기 문항 수: 10
선택된 문항 수: 6
alpha: 0.815 → 0.761

축소된 전체 척도 문항 수: 30
축소 전 전체 크론바흐 알파 평균 : 0.856
축소 후  크론바흐 알파 평균: 0.833


In [ ]:
df_reduced.to_csv('/content/drive/MyDrive/학부연구생/C프로젝트/C프로젝트 최종정리/Second_CTT_reduced_df.csv')